# Solving Elliptic Curve Discrete Logarithm Problem with Shor's Algorithm

## 1. Introduction and Problem Statement

The **Elliptic Curve Discrete Logarithm Problem (ECDLP)** is a fundamental cryptographic challenge that underlies the security of elliptic curve cryptography (ECC). While classical algorithms require exponential time to solve ECDLP, Shor's quantum algorithm can solve it efficiently in polynomial time.


### Elliptic Curve Definition

An elliptic curve over a field $K$, of characteristic different from 2 and 3, is a projective, non-singular curve of genus 1. It can be defined as the locus of points $(x, y) \in K \times K$ satisfying the Weierstrass equation:

$$E : y^2 = x^3 + ax + b$$

where $a, b \in K$ are constants. The curve is projective since it contains the point at infinity, $\mathcal{O}$. The curve is non-singular if $4a^3 + 27b^2 \neq 0$, which ensures that it doesn't have cusps or nodes. The curve is symmetric with respect to the x-axis, a property easily verified by applying the transformation $y \mapsto -y$.

For cryptographic applications, including digital signatures used in Bitcoin, the field of interest is the finite field $\mathbb{F}_p$ for $p > 3$ a prime number. The set of points consisting of $\mathcal{O}$ and all solutions $(x, y) \in \mathbb{F}_p \times \mathbb{F}_p$ to the Weierstrass equation is denoted by:

$$E(\mathbb{F}_p) = \{(x, y) \in \mathbb{F}_p \times \mathbb{F}_p \mid y^2 = x^3 + ax + b\} \cup \{\mathcal{O}\}$$

The set $E(\mathbb{F}_p)$ forms an abelian group with respect to a group operation "+", called addition, that is defined via rational functions in the point coordinates with $\mathcal{O}$ as the neutral element.


### Our Specific Example: $E(\mathbb{F}_7)$

We'll work with a concrete example on a small finite field to demonstrate the concepts, following the approach outlined in Roetteler et al. [[1]](#ref1):

**Elliptic Curve**: $y^2 = x^3 + 5x + 4 \pmod{7}$

**Problem**: Find the discrete logarithm $l$ such that:
$$l \cdot G = P_{target}$$

Where:
- **Generator point** $G = [0, 5]$ 
- **Target point** $P_{target} = [0, 2]$

This is the classical discrete logarithm problem: given points $G$ and $P_{target}$ on the elliptic curve, find the integer $l$ such that $P_{target} = l \cdot G$ (where $l \cdot G$ means adding $G$ to itself $l$ times).

**Solution**: $l = 4$, since $4 \cdot [0, 5] = [0, 2]$





## 2. Shor's Algorithm for ECDLP
                                                                    
Shor's algorithm solves ECDLP by finding relationships of the form:
$$P_0 + x_1 \cdot G - x_2 \cdot P_{target} = \mathcal{O}$$

where $\mathcal{O}$ is the point at infinity, $P_0$ is an auxiliary starting point, and $(x_1, x_2)$ are found using quantum period-finding. From this relationship, the discrete logarithm can be extracted.

**Algorithm Parameters**:
- **Generator point** $G = [0, 5]$ (order 5)
- **Starting point** $P_0 = [4, 2]$ (auxiliary point for the algorithm)  
- **Target point** $P_{target} = [0, 2]$
- **Goal**: Find $l$ such that $l \cdot G = P_{target}$

### Mathematical Foundation

The algorithm exploits the periodicity of the function $f(x_1, x_2) = P_0 + x_1 \cdot G - x_2 \cdot P_{target}$ in quantum superposition. When this function evaluates to the point at infinity $\mathcal{O}$, we have:

$$P_0 + x_1 \cdot G = x_2 \cdot P_{target}$$

Since $P_{target} = l \cdot G$ (by definition of the discrete logarithm), this becomes:
$$P_0 + x_1 \cdot G = x_2 \cdot l \cdot G$$

Rearranging: $(x_1 - x_2 \cdot l) \cdot G = -P_0$

**Why is this function periodic?** The key insight is that if $(x_1, x_2)$ is a solution (i.e., $f(x_1, x_2) = \mathcal{O}$), then $(x_1 + kr, x_2 + k)$ is also a solution for any integer $k$, where $r$ is the order of the generator $G$.

This periodicity arises because:
$$f(x_1 + kr, x_2 + k) = P_0 + (x_1 + kr) \cdot G - (x_2 + k) \cdot P_{target}$$
$$= P_0 + x_1 \cdot G + kr \cdot G - x_2 \cdot P_{target} - k \cdot P_{target}$$
$$= P_0 + x_1 \cdot G - x_2 \cdot P_{target} + k(r \cdot G - P_{target})$$

Since $r \cdot G = \mathcal{O}$ (by definition of order) and $P_{target} = l \cdot G$:
$$= f(x_1, x_2) + k(\mathcal{O} - l \cdot G) = f(x_1, x_2) - kl \cdot G$$

When $f(x_1, x_2) = \mathcal{O}$, we get $f(x_1 + kr, x_2 + k) = -kl \cdot G$, which equals $\mathcal{O}$ when $kl \equiv 0 \pmod{r}$.

**Quantum Period Finding**: The Quantum Fourier Transform extracts this hidden period structure from the superposition of all $(x_1, x_2)$ pairs, revealing relationships that allow us to solve for the discrete logarithm $l$ using continued fractions and modular arithmetic.

From the quantum measurement results, we can extract the discrete logarithm $l$ using continued fractions and modular arithmetic.

### Curve Properties

The elliptic curve $y^2 = x^3 + 5x + 4 \pmod{7}$ contains the following points:
- $[0, 2], [0, 5], [2, 1], [2, 6], [3, 2], [3, 5], [4, 2], [4, 5], [5, 0]$
- Plus the point at infinity $\mathcal{O}$

**Generator $G = [0, 5]$ properties**:
- Order: 5 (since $5 \cdot G = \mathcal{O}$)
- Multiples: $1 \cdot G = [0, 5]$, $2 \cdot G = [2, 1]$, $3 \cdot G = [2, 6]$, $4 \cdot G = [0, 2]$, $5 \cdot G = \mathcal{O}$

### Algorithm Complexity and Quantum Advantage

**Classical Complexity**: The best known classical algorithms for ECDLP (such as Pollard's rho algorithm) require $O(\sqrt{n})$ operations, where $n$ is the order of the elliptic curve group. For cryptographically relevant curves with 256-bit keys, this means approximately $2^{128}$ operations.

**Quantum Complexity**: Shor's algorithm reduces this to $O((\log n)^3)$ operations using $O(\log n)$ qubits, providing an exponential speedup. For the same 256-bit curves, this reduces to approximately $2^{24}$ operations.

This exponential quantum advantage is what makes Shor's algorithm a potential threat to elliptic curve cryptography used in modern security systems, including:
- TLS/SSL certificates
- Bitcoin and cryptocurrency wallets  
- Digital signatures
- Key exchange protocols


### Quantum Algorithm Steps

1. **Superposition Creation**: Create quantum superposition over all possible values of $k_1$ and $k_2$
2. **Quantum Arithmetic**: Perform elliptic curve computation $P_0 + k_1 \cdot G - k_2 \cdot P_{target}$ in superposition
3. **Period Extraction**: Use Quantum Fourier Transform to extract period information
4. **Classical Post-processing**: Analyze measurement results to find the discrete logarithm



### Problem Setup

Let's define our specific ECDLP instance and import the necessary Classiq quantum computing tools. The following code sets up the elliptic curve parameters and global parameters for our implementation.

In [1]:
# Import required libraries
import ast
import re
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np

from classiq import (
    CX,
    SWAP,
    Constraints,
    Output,
    Preferences,
    QArray,
    QBit,
    QNum,
    X,
    allocate,
    apply_to_all,
    bind,
    control,
    create_model,
    execute,
    free,
    hadamard_transform,
    inplace_add,
    invert,
    qft,
    qfunc,
    show,
    synthesize,
    within_apply,
)
from classiq.qmod import SIGNED
from classiq.qmod.symbolic import ceiling, log


# Define our elliptic curve and problem parameters
class EllipticCurve:
    def __init__(self, p, a, b):
        """
        Represents an elliptic curve of the form y^2 = x^3 + a*x + b (mod p)
        """
        self.p = p
        self.a = a
        self.b = b

    def __repr__(self):
        return f"EllipticCurve(p={self.p}, a={self.a}, b={self.b})"


# Problem parameters for our specific ECDLP instance
CURVE = EllipticCurve(p=7, a=5, b=4)
GENERATOR_G = [0, 5]  # Generator point G
INITIAL_POINT = [4, 2]  # Starting point P₀
TARGET_POINT = [0, 2]  # Target point P_target

### Main Quantum Function

The core of our implementation is the `main_shor_ecdlp` quantum function that orchestrates the entire algorithm.

### Expected Quantum Flow

1. **Initialization** : All variables start in $|0\rangle$ state
2. **State Preparation** : Set initial elliptic curve point to $[4, 2]$
3. **Superposition Creation** : Apply Hadamard transforms to create $\frac{1}{8}\sum_{k_1,k_2=0}^{7} |k_1\rangle|k_2\rangle$
4. **Quantum Arithmetic** : Perform elliptic curve operations in superposition
5. **Period Extraction** : Apply inverse QFT to extract period information
6. **Measurement** : Collapse superposition and read classical outcomes

In [2]:
# Main Shor ECDLP quantum function from shor_ec.py
from classiq import (
    Constraints,
    Output,
    Preferences,
    QArray,
    QBit,
    QNum,
    allocate,
    create_model,
    execute,
    hadamard_transform,
    invert,
    qft,
    qfunc,
    show,
    synthesize,
)
from classiq.qmod.symbolic import ceiling, log


@qfunc
def main_shor_ecdlp(
    ecp_x: Output[QNum],  # x-coordinate of elliptic curve point
    ecp_y: Output[QNum],  # y-coordinate of elliptic curve point
    x1: Output[QNum],  # first quantum register for period finding
    x2: Output[QNum],  # second quantum register for period finding
) -> None:
    """
    Main quantum function implementing Shor's algorithm for ECDLP.

    Step-by-step breakdown:
    1. Initialize quantum variables
    2. Prepare initial elliptic curve point P₀
    3. Create superposition over x1 and x2 values
    4. Perform quantum elliptic curve arithmetic: P₀ + x1·G - x2·P_target
    5. Apply inverse quantum Fourier transform for period extraction
    """

    # Step 1: Allocate quantum registers for the EC point and QPE variables
    n = CURVE.p.bit_length()  # Number of bits needed for our small field (mod 7)
    allocate(n, ecp_x)  # for x-coordinate
    allocate(n, ecp_y)  # for y-coordinate
    allocate(n, x1)  # for first period-finding register
    allocate(n, x2)  # for second period-finding register

    # Step 2: Initialize ecp to INITIAL_POINT [4, 2]
    ecp_x ^= INITIAL_POINT[0]  # Set initial x-coordinate
    ecp_y ^= INITIAL_POINT[1]  # Set initial y-coordinate

    # Step 3: Create uniform superposition on x1 and x2
    hadamard_transform(x1)
    hadamard_transform(x2)

    # Step 4: Quantum elliptic curve arithmetic in superposition
    # First: ecp = P₀ + x1·G (add x1 copies of generator G)
    ell_mult_add(ecp_x, ecp_y, x1, GENERATOR_G, CURVE.p, CURVE.a, CURVE.b)

    # Second: ecp = P₀ + x1·G - x2·P_target (subtract x2 copies of target)
    # To subtract x2·P_target, we add x2·(-P_target)
    neg_target = [TARGET_POINT[0], (-TARGET_POINT[1]) % CURVE.p]
    ell_mult_add(ecp_x, ecp_y, x2, neg_target, CURVE.p, CURVE.a, CURVE.b)

    # Step 5: Inverse Quantum Fourier Transform for period extraction
    # This extracts the period information from the quantum registers
    invert(lambda: qft(x1))
    invert(lambda: qft(x2))

### Quantum Variables Architecture

Our implementation uses 6 quantum variables:

| Var | Qubits | Purpose |
|----------|--------|---------|
| `ecp_x` | 3 | x-coordinate of elliptic curve point during computation |
| `ecp_y` | 3 | y-coordinate of elliptic curve point during computation |
| `t0` | 3 | Auxiliary register for temporary storage and modular inverse |
| `l` | 3 | Lambda register for storing slope calculations in point addition |
| `x1` | 3 | First period-finding register (ranges 0-7) |
| `x2` | 3 | Second period-finding register (ranges 0-7) |

The algorithm creates a superposition over $2^6 = 64$ different combinations of $(k_1, k_2)$ values and evaluates the elliptic curve expression simultaneously.

## 3. Quantum Elliptic Curve Addition

### Algorithm Hierarchy Overview

The `main_shor_ecdlp` function orchestrates Shor's algorithm, but its computational core is the `ell_mult_add` function, which performs quantum scalar multiplication in superposition. This function computes expressions like $P_0 + k \cdot G$ where $k$ is in quantum superposition, enabling the algorithm to evaluate all possible scalar values simultaneously.

At the heart of `ell_mult_add` lies the fundamental building block: **Quantum Point Addition (`ec_point_add`)** - the reversible quantum implementation of classical elliptic curve point addition. The `ell_mult_add` function uses the double-and-add algorithm to compute scalar multiples $k \cdot P$ by repeatedly calling `ec_point_add` in a controlled manner based on the binary representation of $k$.

### Quantum Point Addition (`ec_point_add`)

The `ec_point_add` function performs the fundamental operation of adding two elliptic curve points in a quantum-reversible manner. This is the atomic operation that enables all higher-level elliptic curve arithmetic.

**Complete Elliptic Curve Addition Cases**:

In general, elliptic curve point addition must handle several cases:
1. **Generic case**: Two distinct points $P \neq Q$ where $P \neq -Q$
  - Slope: $\lambda = \frac{y_Q - y_P}{x_Q - x_P} \pmod{p}$
2. **Point doubling**: Adding a point to itself $P + P$
  - Slope: $\lambda = \frac{3x_P^2 + a}{2y_P} \pmod{p}$
3. **Inverse points**: $P + (-P) = \mathcal{O}$ (point at infinity)
4. **Adding point at infinity**: $P + \mathcal{O} = P$

**Our Implementation Simplification**:

Following the approach in Roetteler et al. [17] and Gouzien et al. [7], our implementation focuses only on the **generic case** where the two points are distinct, not inverses of each other, and neither is the neutral element. These exceptional cases are rare, except when the accumulation register is initialized to the neutral element.

To avoid edge cases entirely, we initialize the accumulation register with a non-zero point ($P_0 = [4, 2]$) rather than the neutral element. This strategy does not affect the measurement statistics after the Quantum Fourier Transform, since it only adds a global phase to the final quantum state.

**Algorithm Overview** (Generic Case Only):
1. **Input**: Quantum point $(x, y)$ and classical point $G = (G_x, G_y)$
2. **Slope calculation**: Compute $\lambda = \frac{y - G_y}{x - G_x} \pmod{p}$ using quantum modular inverse
3. **Result coordinates**: 
  - $x_{result} = \lambda^2 - x - G_x \pmod{p}$
  - $y_{result} = \lambda(x - x_{result}) - y \pmod{p}$
4. **Cleanup**: Uncompute auxiliary values to maintain reversibility

**Key Quantum Challenge**: All operations must be reversible (unitary) and use auxiliary qubits for temporary storage, making this significantly more complex than classical elliptic curve addition.

**Reference**: The `ec_point_add` function is based on **Algorithm 1** from *"Quantum resource estimates for computing elliptic curve discrete logarithms"* by Roetteler et al. (2017) [[1]](#ref1)

In [3]:
# Quantum elliptic curve point addition function
from classiq import (
    CInt,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def ec_point_add(
    x: QNum,  # x-coordinate of quantum point
    y: QNum,  # y-coordinate of quantum point
    G: list[int],  # Classical point coordinates [Gx, Gy] on the curve
    p: int,  # Prime modulus
) -> None:
    """
    Performs in-place elliptic curve point addition of a point whose coordinates are
    stored in quantum registers and a classically known point; the result of the
    operation is stored in the registers initially containing the coordinates of the
    input.

    Algorithm (Elliptic Curve Point Addition):
    1. Compute slope λ = (y - Gy) / (x - Gx) mod p
    2. Compute new x-coordinate: x_result = λ² - x - Gx mod p
    3. Compute new y-coordinate: y_result = λ(x - x_result) - y mod p
    4. Clean up auxiliary registers to maintain reversibility

    Args:
        x: Quantum register containing the x-coordinate of the quantum point.
           The result x-coordinate will be stored in-place here.
        y: Quantum register containing the y-coordinate of the quantum point.
           The result y-coordinate will be stored in-place here.
        G: List of 2 classical coordinates [Gx, Gy] representing a point on the curve.
        p: Prime modulus for the elliptic curve operations.
    """

    l = QNum("l")  # aux quantum register for lambda (slope)
    allocate(CURVE.p.bit_length(), l)
    t0 = QNum("t0")  # aux quantum register
    allocate(CURVE.p.bit_length(), t0)
    temp = QNum("temp")  # aux quantum register
    allocate(CURVE.p.bit_length(), temp)

    # Extract classical coordinates
    Gx = G[0]  # x-coordinate of classical point
    Gy = G[1]  # y-coordinate of classical point

    # Step 1: Compute y - Gy mod p
    modular_in_place_subtract_constant(y, Gy, p)  # y becomes (y - Gy) mod p

    # Step 2: Compute x - Gx mod p
    modular_in_place_subtract_constant(x, Gx, p)  # x becomes (x - Gx) mod p

    # Step 3: Compute slope λ = (y - Gy) / (x - Gx) mod p
    # This uses the quantum modular inverse and multiplication
    within_apply(
        lambda: mock_kaliski_inverse_modulus_7(x, t0),  # t0 = x^(-1) mod p
        lambda: modular_out_of_place_multiply(t0, y, l, p),  # l = t0 * y = λ mod p
    )

    # Step 4: Compute x_result = λ² - x - Gx mod p
    within_apply(
        lambda: modular_out_of_place_multiply(l, x, t0, p),  # t0 = λ * x
        lambda: modular_in_place_subtract(t0, y, p),  # y = t0 - y = λx - y
    )

    # Step 5: Compute final x-coordinate
    within_apply(
        lambda: modular_square(l, t0, p),  # t0 = λ²
        lambda: (
            modular_in_place_subtract(t0, x, p),  # x = t0 - x = λ² - x
            modular_in_place_negate(x, p),  # x = -x mod p
            modular_in_place_add_constant(x, (3 * Gx) % p, p),  # x = x + 3*Gx mod p
        ),
    )

    # Step 6: Compute final y-coordinate
    modular_out_of_place_multiply(l, x, y, p)  # y = λ * x

    # Step 7: Clean up lambda register to maintain reversibility
    within_apply(
        lambda: mock_kaliski_inverse_modulus_7(x, t0),  # Recompute inverse
        lambda: within_apply(
            lambda: modular_out_of_place_multiply(t0, y, temp, p),  # temp = t0 * y
            lambda: modular_in_place_subtract(temp, l, p),  # l = temp - l (cleanup)
        ),
    )

    # Step 8: Final coordinate adjustments
    modular_in_place_subtract_constant(y, Gy, p)  # Adjust y-coordinate
    modular_in_place_negate(x, p)  # Negate x
    modular_in_place_add_constant(x, Gx, p)  # Add back Gx

    free(l)
    free(t0)
    free(temp)

### Quantum Scalar Multiplication (`ell_mult_add`)

This implements the **double-and-add algorithm** in quantum superposition:

**Algorithm**: For scalar $k$ in binary representation $k = \sum_{i=0}^{n-1} k_i \cdot 2^i$:
1. Start with accumulator point initialized to input point
2. For each bit $k_i$ from LSB to MSB:
   - If $k_i = 1$: add current power of base point to accumulator
   - Double the current power of base point for next iteration

**Quantum Advantage**: All possible values of $k$ are processed simultaneously in superposition!

When we have $|k_1\rangle|k_2\rangle$ in superposition, the quantum circuit evaluates:
$$\frac{1}{8} \sum_{k_1,k_2=0}^{7} |k_1\rangle|k_2\rangle|P_0 + k_1 \cdot G - k_2 \cdot P_{target}\rangle$$

In [4]:
# Quantum scalar multiplication function
from classiq import (
    CInt,
    Output,
    QArray,
    QBit,
    QNum,
    X,
    allocate,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def ell_mult_add(
    x: QNum,  # x-coordinate of accumulator point
    y: QNum,  # y-coordinate of accumulator point
    k: QArray[QBit],  # scalar in binary representation
    P: list[int],  # classical point to multiply [x, y]
    p: int,  # prime modulus
    a: int,
    b: int,  # curve parameters
) -> None:
    """
    Quantum scalar multiplication: computes k*P and adds to (x,y) in-place.

    Algorithm (Double-and-Add in Quantum Superposition):
    This implements the classical double-and-add algorithm but operates on all
    possible values of k simultaneously when k is in quantum superposition.

    For scalar k = Σᵢ kᵢ·2ⁱ in binary representation:
    1. Initialize: current_power = P (represents 2⁰·P = P)
    2. For each bit kᵢ from LSB to MSB:
       - If kᵢ = 1: add current_power to accumulator (controlled by kᵢ)
       - Update: current_power = 2·current_power (classical doubling for next iteration)
    3. Result: accumulator contains original_point + k·P

    Quantum Advantage: When k is in superposition |k⟩ = Σₖ αₖ|k⟩, this computes
    all possible scalar multiplications simultaneously:
    Σₖ αₖ|k⟩|(x,y) + k·P⟩

    Args:
        x: x-coordinate of accumulator point (updated in-place)
        y: y-coordinate of accumulator point (updated in-place)
        k: quantum bit array representing scalar in binary (LSB to MSB)
        P: classical point [Px, Py] to be multiplied by k
        p: prime modulus of the elliptic curve field
        a, b: elliptic curve parameters for y² = x³ + ax + b
    """

    n = k.size  # Number of bits in scalar k
    current_power = P.copy()  # Start with 1·P = P

    # Process each bit of k from LSB (bit 0) to MSB (bit n-1)
    for i in range(n):

        # Controlled point addition: if k[i] = 1, add current_power to accumulator
        # This is the core quantum operation - controlled by qubit k[i]
        control(k[i], lambda: ec_point_add(x, y, current_power, p))

        # Classical update for next iteration: current_power = 2 * current_power
        # This represents the next power of 2 in the binary expansion
        if i < n - 1:  # Don't double after the last iteration
            curve = EllipticCurve(p=p, a=a, b=b)
            current_power = ell_double_classical(current_power, curve)


def ell_double_classical(P, curve):
    """
    Classical elliptic curve point doubling for updating powers in ell_mult_add.
    Returns 2P for a point P on the elliptic curve.
    """
    p = curve.p
    x, y = P
    # Slope calculation: s = (3*x² + a) / (2*y) mod p
    numerator = (3 * (x * x % p) + curve.a) % p
    denominator = (2 * y) % p
    s = (numerator * pow(denominator, -1, p)) % p
    # x-coordinate of the result
    xr = (s * s - 2 * x) % p
    # y-coordinate of the result
    yr = (y - s * ((x - xr) % p)) % p
    # Return the result, with y in the standard form
    return [xr, (p - yr) % p]

## 4. Modular Arithmetic Building Blocks

To implement `ec_point_add`, we need a complete library of reversible modular arithmetic operations. These functions form the foundation of quantum elliptic curve arithmetic:

### Basic Operations
- `modular_in_place_add(x, y, modulus)` - Computes $(x + y) \bmod p$ in-place on $y$
- `modular_in_place_subtract(x, y, modulus)` - Computes $(x - y) \bmod p$ in-place on $y$  
- `modular_in_place_negate(x, modulus)` - Computes $(-x) \bmod p$ in-place on $x$

### Constant Operations  
- `modular_in_place_add_constant(x, c, modulus)` - Computes $(x + c) \bmod p$ in-place on $x$
- `modular_in_place_subtract_constant(x, c, modulus)` - Computes $(x - c) \bmod p$ in-place on $x$

### Advanced Operations
- `modular_out_of_place_multiply(x, y, z, modulus)` - Computes $z = (x \times y) \bmod p$
- `modular_square(x, z, p)` - Computes $z = x^2 \bmod p$
- `modular_in_place_double(x, modulus)` - Computes $2x \bmod p$ in-place on $x$
- `modular_inverse` - Computes $x^{-1} \bmod p$

**Note on Modular Inverse**: For simplicity, we use a mock function `mock_kaliski_inverse_modulus_7` in our implementation. A rigorous implementation using techniques like the extended Euclidean algorithm will be covered in the next tutorial.

### Quantum Arithmetic Principles

All modular arithmetic functions must satisfy these quantum computing requirements:
1. **Reversibility**: All operations must be unitary (reversible)
2. **Auxiliary Qubits**: Use temporary qubits for intermediate calculations
3. **Compute-Uncompute Pattern**: Clean up auxiliary qubits to avoid entanglement

Let's examine each function and synthesize quantum circuits to understand their implementation.

We will use the utility function `run_quantum_test` to view and execute each quantum program:

In [5]:
def run_quantum_test(main_func, test_name: str) -> None:
    """
    Helper function to run a quantum test with optimization constraints and print results.

    Args:
        main_func: The main quantum function to be tested
        test_name: Name of the test for display purposes

    Returns:
        Parsed counts from quantum execution
    """
    # Set up optimization constraints
    constraints = Constraints(
        optimization_parameter="width",  # Optimize for minimum width
    )
    preferences = Preferences(timeout_seconds=3600, optimization_level=1)

    # Create model with constraints
    qmod = create_model(main_func, constraints=constraints, preferences=preferences)

    # Synthesize quantum circuit
    print("Synthesizing...")
    qprog = synthesize(qmod)

    # Display circuit (uncomment if needed)
    show(qprog)

    # Display circuit metrics
    print(f"\n🔧 {test_name} Circuit Metrics:")
    print(f"  Width (qubits): {qprog.data.width}")

    # Execute quantum circuit
    print("Executing...")
    result = execute(qprog).result()

    # Display results
    print(f"\n📊 {test_name} Test Results:")
    print("Quantum Result:", result[0].value.parsed_counts)

    return result[0].value.parsed_counts

### Modular In-Place Addition

Computes $(x + y) \bmod p$ and stores the result in-place in register $y$.

In [6]:
# modular_in_place_add implementation and circuit synthesis
from classiq import (
    SIGNED,
    CInt,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def modular_in_place_add(x: QNum, y: QNum, modulus: int) -> None:
    """
    Reversible, in-place modular addition of two integers modulo a constant integer modulus.
    Given two n-bit integers x and y encoded in quantum registers `x` and `y`, and a constant integer `modulus`,
    this function computes the sum (x + y) mod modulus in a reversible manner.
    The result is stored in-place in the `y` register.

    Args:
        x (QNum): Quantum register encoding the first integer addend (x).
        y (QNum): Quantum register encoding the second integer addend (y).
                  This register will be updated in-place to hold the modular sum.
        modulus (int): Constant integer modulus for the modular addition.
    """
    # Use a carry qubit to detect overflow
    carry = QBit("carry")
    allocate(1, carry)

    # Create temporary register that combines y and carry
    res_and_carry = QNum("res_and_carry", y.size + 1, SIGNED, 0)

    # Compute-uncompute pattern for reversible arithmetic
    within_apply(
        # COMPUTE: Bind y and carry into combined register
        lambda: bind([y, carry], res_and_carry),
        # APPLY: Perform modular arithmetic operations
        lambda: (
            inplace_add(x, res_and_carry),  # Add x to the combined register
            inplace_add(-modulus, res_and_carry),  # Subtract modulus to check overflow
        ),
    )
    # UNCOMPUTE: y and carry are automatically separated

    # If carry is set (negative result), add modulus back to y
    control(carry, lambda: inplace_add(modulus, y))

    # Update carry qubit based on comparison (y >= x after operation)
    control(y >= x, lambda: X(carry))

    # Clean up auxiliary qubit
    free(carry)

**Demonstrating the modular addition**: Perform modular addition:
$$y = (x + y) \bmod 7 = (3 + 4) \bmod 7 = 0$$

In [7]:
@qfunc
def main(x: Output[QNum], y: Output[QNum]) -> None:
    """Main function for modular addition test"""
    allocate(3, x)  # 3 qubits for mod 7 arithmetic
    allocate(3, y)  # 3 qubits for mod 7 arithmetic

    # Initialize with test values: x=3, y=4
    x ^= 3
    y ^= 4

    # Perform modular addition: y = (x + y) mod 7 = (3 + 4) mod 7 = 0
    modular_in_place_add(x, y, CURVE.p)

In [8]:
run_quantum_test(main, "modular_in_place_add")

Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVbZiHtw5ljx9veURpbAHqOBiE

🔧 modular_in_place_add Circuit Metrics:
  Width (qubits): 9
Executing...

📊 modular_in_place_add Test Results:
Quantum Result: [{'x': 3, 'y': 0}: 2048]


[{'x': 3, 'y': 0}: 2048]

### Modular In-Place Negation

Computes $(-x) \bmod p$ - finds the additive inverse.


*Remark: Uses the trick that $x-y = (x'+y)'$, where ' denotes the one's complement.*


In [9]:
# modular_in_place_negate implementation and circuit synthesis
from classiq import (
    SIGNED,
    CInt,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    apply_to_all,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def check_if_all_zero(x: QArray[QBit], ancilla: QBit) -> None:
    """
    Sets the ancilla qubit to 1 if all qubits in the quantum array `x` are zero, otherwise sets it to 0.
    Uses within_apply to handle the compute/uncompute of X gates.
    """
    within_apply(lambda: apply_to_all(X, x), lambda: control(x, lambda: X(ancilla)))


@qfunc
def check_if_all_ones(x: QArray[QBit], ancilla: QBit) -> None:
    """
    Sets the ancilla qubit to 1 if all qubits in the quantum array `x` are one, otherwise sets it to 0.
    """
    control(x, lambda: X(ancilla))


@qfunc
def bitwise_negation(arr: QArray[QBit]) -> None:
    """
    Applies an X gate (bitwise NOT) to each qubit in the quantum array `arr`.
    """
    apply_to_all(X, arr)


@qfunc
def modular_in_place_negate(x: QNum, modulus: int) -> None:
    """
    Reversible, in-place modular negation of an integer modulo a constant integer modulus.
    Given an n-bit integer x encoded in quantum register `x`, and a constant integer `modulus`,
    this function computes (-x) mod modulus and stores the result in-place in `x`.

    Algorithm:
    1. Check if x is all zeros (special case)
    2. If x=0, set x to modulus, otherwise compute complement
    3. Apply bitwise negation to get final result

    Args:
        x (QNum): Quantum numeric register to be negated in-place.
        modulus (int): Constant integer modulus.
    """
    n = x.size
    neg_modulus = 2**n - modulus - 1

    is_all_zeros = QBit("is_all_zeros")
    allocate(1, is_all_zeros)

    # Test if the input is all-zeros
    check_if_all_zero(x, is_all_zeros)

    # If all-zeros, then put the modulus in x
    control(is_all_zeros, lambda: inplace_add(modulus, x))

    # Add neg_modulus to x
    inplace_add(neg_modulus, x)

    # If x=0, then we have neg_modulus + modulus = all ones
    check_if_all_ones(x, is_all_zeros)

    # Bitwise negation: set x to (neg_modulus + x)'
    bitwise_negation(x)

    free(is_all_zeros)

**Demonstrating the modular negation**: Compute modular negation:
$$x \leftarrow (-x) \bmod 7 = (-5) \bmod 7 = 2$$

In [10]:
@qfunc
def main(x: Output[QNum]) -> None:
    """Main function for modular negation test"""
    allocate(3, x)  # 3 qubits for mod 7 arithmetic

    # Initialize with test value: x=5
    x ^= 5

    # Perform modular negation: x = (-x) mod 7 = (-5) mod 7 = 2
    modular_in_place_negate(x, CURVE.p)

In [11]:
run_quantum_test(main, "modular_in_place_negate")

Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVbaeD14it1gUc5zLThWc0F7wX

🔧 modular_in_place_negate Circuit Metrics:
  Width (qubits): 4
Executing...

📊 modular_in_place_negate Test Results:
Quantum Result: [{'x': 2}: 2048]


[{'x': 2}: 2048]

### Modular In-Place Subtraction

Computes $(x - y) \bmod p$ and stores the result in-place in register $y$.

In [12]:
# modular_in_place_subtract implementation and circuit synthesis
from classiq import (
    SIGNED,
    CInt,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    apply_to_all,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def modular_in_place_subtract(x: QNum, y: QNum, modulus: int) -> None:
    """
    Reversible, in-place modular subtraction of two integers modulo a constant integer modulus.
    Given two n-bit integers x and y encoded in quantum registers `x` and `y`, and a constant integer `modulus`,
    this function computes the difference (x - y) mod modulus in a reversible manner.
    The result is stored in-place in the `y` register.

    Algorithm:
    1. Compute (-y) mod modulus in-place on y
    2. Compute x + (-y) mod modulus in-place on y

    Args:
        x (QNum): Quantum register encoding the first integer operand (x).
        y (QNum): Quantum register encoding the second integer operand (y).
                  This register will be updated in-place to hold the modular difference.
        modulus (int): Constant integer modulus for the modular subtraction.
    """
    # Compute -y mod modulus in-place on y
    modular_in_place_negate(y, modulus)
    # Compute x + (-y) mod modulus in-place on y
    modular_in_place_add(x, y, modulus)

**Demonstrating the modular subtraction**: Compute modular subtraction:
$$y \leftarrow (x - y) \bmod 7 = (6 - 4) \bmod 7 = 2$$

**Algorithm steps**:
1. First: $y \leftarrow (-y) \bmod 7 = (-4) \bmod 7 = 3$
2. Then: $y \leftarrow (x + y) \bmod 7 = (6 + 3) \bmod 7 = 2$

In [13]:
@qfunc
def main(x: Output[QNum], y: Output[QNum]) -> None:
    """Main function for modular subtraction test"""
    allocate(3, x)  # 3 qubits for mod 7 arithmetic
    allocate(3, y)  # 3 qubits for mod 7 arithmetic

    # Initialize with test values: x=6, y=4
    x ^= 6
    y ^= 4

    # Perform modular subtraction: y = (x - y) mod 7 = (6 - 4) mod 7 = 2
    modular_in_place_subtract(x, y, CURVE.p)

In [14]:
run_quantum_test(main, "modular_in_place_subtract")

Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVbbtpNjdHgz2lMDMcgS8cssZB

🔧 modular_in_place_subtract Circuit Metrics:
  Width (qubits): 9
Executing...

📊 modular_in_place_subtract Test Results:
Quantum Result: [{'x': 6, 'y': 2}: 2048]


[{'x': 6, 'y': 2}: 2048]

### Modular Addition with Constant

Computes $(x + c) \bmod p$ where $c$ is a classical constant.

In [15]:
# modular_in_place_add_constant implementation and circuit synthesis
from classiq import (
    SIGNED,
    CInt,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def modular_in_place_add_constant(x: QNum, constant: int, modulus: int) -> None:
    """
    Computes (x + constant) mod modulus in-place on x.
    Uses a single carry qubit to check if we need to subtract modulus.

    This function demonstrates key quantum arithmetic techniques:
    1. Auxiliary qubits for overflow detection
    2. Compute-uncompute patterns for reversibility
    3. Conditional modular reduction

    Args:
        x (QNum): The quantum register to add to.
        constant (int): The classical integer constant to add.
        modulus (int): The integer modulus.
    """
    # Allocate a single carry qubit
    carry = QBit("carry")
    allocate(1, carry)

    # Create and allocate temporary register to hold x + carry
    temp = QNum("temp", x.size + 1, SIGNED, 0)

    # Bind x and carry into temp for addition
    within_apply(
        lambda: bind([x, carry], temp),
        lambda: (
            # Add constant to temp
            inplace_add(constant, temp),
            # Add -modulus to temp to check for overflow
            inplace_add(-modulus, temp),
        ),
    )

    # If carry is set, we need to add modulus back
    control(carry, lambda: inplace_add(modulus, x))
    control(x >= constant, lambda: X(carry))

    # Free the temporary registers
    free(carry)

**Demonstrating modular addition with constant**: Add classical constant to quantum register:
$$x \leftarrow (x + c) \bmod 7 = (2 + 3) \bmod 7 = 5$$

**Algorithm steps**:
1. **Setup**: $x = 2$, constant $c = 3$, modulus $= 7$
2. **Temporary calculation**: $temp = x + c - modulus = 2 + 3 - 7 = -2$
3. **Overflow check**: Since $temp < 0$, carry bit is set
4. **Correction**: Add modulus back: $x = 2 + 3 = 5$
5. **Result**: $x = 5$

This operation is particularly useful in elliptic curve arithmetic where we frequently add curve parameters (like coefficients $a$ and $b$) to quantum coordinates.

In [16]:
@qfunc
def main(x: Output[QNum]) -> None:
    """Main function for modular addition with constant test"""
    allocate(3, x)  # 3 qubits for mod 7 arithmetic

    # Initialize with test value: x=2
    x ^= 2

    # Perform modular addition with constant: x = (x + 3) mod 7 = (2 + 3) mod 7 = 5
    modular_in_place_add_constant(x, 3, CURVE.p)

In [17]:
# Execute the quantum test
result = run_quantum_test(main, "Modular Add Constant")

Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVbcr6soSE7Idwhg3TPaFmCxzH

🔧 Modular Add Constant Circuit Metrics:
  Width (qubits): 6
Executing...

📊 Modular Add Constant Test Results:
Quantum Result: [{'x': 5}: 2048]


### Modular Subtraction with Constant

Computes $(x - c) \bmod p$ where $c$ is a classical constant.

In [18]:
# modular_in_place_subtract_constant implementation and circuit synthesis
from classiq import (
    SIGNED,
    CInt,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def modular_in_place_subtract_constant(x: QNum, constant: int, modulus: int) -> None:
    """
    Computes (x - constant) mod modulus in-place on x.
    Uses modular_in_place_add_constant with negated constant.

    This function demonstrates:
    1. Reduction of subtraction to addition with negative constant
    2. Efficient reuse of existing quantum arithmetic functions
    3. Modular arithmetic properties: (x - c) ≡ (x + (-c mod p)) (mod p)

    Args:
        x (QNum): The quantum register to subtract from.
        constant (int): The classical integer constant to subtract.
        modulus (int): The integer modulus.
    """
    # Compute -constant mod modulus and use add_constant
    neg_const = (-constant) % modulus
    modular_in_place_add_constant(x, neg_const, modulus)

**Demonstrating modular subtraction with constant**: Subtract classical constant from quantum register:
$$x \leftarrow (x - c) \bmod 7 = (5 - 3) \bmod 7 = 2$$

In [19]:
@qfunc
def main(x: Output[QNum]) -> None:
    """Main function for modular subtraction with constant test"""
    allocate(3, x)  # 3 qubits for mod 7 arithmetic

    # Initialize with test value: x=5
    x ^= 5

    # Perform modular subtraction with constant: x = (x - 3) mod 7 = (5 - 3) mod 7 = 2
    modular_in_place_subtract_constant(x, 3, CURVE.p)

In [20]:
result = run_quantum_test(main, "Modular Subtract Constant")

Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVbdu8ZR212VUftZmCXEWF5Rfu

🔧 Modular Subtract Constant Circuit Metrics:
  Width (qubits): 5
Executing...

📊 Modular Subtract Constant Test Results:
Quantum Result: [{'x': 2}: 2048]


### Modular In-Place Double

Computes $2x \bmod p$ - efficient doubling using bit shifts.

In [21]:
# modular_in_place_double implementation and circuit synthesis
from classiq import (
    CX,
    SIGNED,
    SWAP,
    CInt,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def shift_left(reg: QArray[QBit]) -> None:
    """
    Performs a logical left shift on the quantum register array `reg` using SWAP gates.
    The most significant bit is set to 0, and all other bits are shifted left by one position.
    This is equivalent to multiplication by 2.
    """
    n = reg.size
    # Shift bits left using SWAP gates
    for i in reversed(range(1, n)):
        SWAP(reg[i], reg[i - 1])


@qfunc
def modular_in_place_double(x: QNum, modulus: int) -> None:
    """
    Doubles the value in the quantum register `x` (computes 2x), then reduces it modulo `modulus`.
    Uses an ancilla qubit `carry` to capture overflow from the doubling operation, and updates
    the carry using the least significant bit of x.

    Algorithm:
    1. Use carry qubit to extend register size
    2. Perform left shift (multiplication by 2)
    3. Subtract modulus to check for overflow
    4. Conditionally add modulus back if needed
    5. Update carry based on LSB analysis

    Args:
        x (QNum): Quantum register to be doubled and reduced modulo `modulus`.
        modulus (int): Constant integer modulus for the modular reduction.
    """
    carry = QBit("carry")
    allocate(1, carry)
    res_and_carry = QNum("res_and_carry", x.size + 1, SIGNED, 0)

    within_apply(
        lambda: bind([x, carry], res_and_carry),
        lambda: (
            shift_left(res_and_carry),  # holds 2*x
            inplace_add(-modulus, res_and_carry),
        ),
    )

    control(carry, lambda: inplace_add(modulus, x))

    # Update carry based on LSB of original x
    lsb = QNum("lsb", 1)
    rest = QNum("rest", x.size - 1)
    within_apply(
        lambda: bind(x, [lsb, rest]),
        lambda: (
            X(lsb),
            CX(lsb, carry),
            X(lsb),
        ),
    )
    free(carry)

**Demonstrating modular doubling**: Double quantum register value:
$$x \leftarrow 2x \bmod 7 = 2 \cdot 3 \bmod 7 = 6$$

**Algorithm steps**:
1. **Setup**: $x = 3$ (binary: $011$), modulus $= 7$
2. **Left shift**: $2x = 6$ (binary: $110$)
3. **Check overflow**: $6 - 7 = -1 < 0$, so carry bit is set
4. **No correction needed**: Since $2x < modulus$, result is $6$
5. **LSB analysis**: Update carry based on original LSB pattern
6. **Result**: $x = 6$

In [22]:
@qfunc
def main(x: Output[QNum]) -> None:
    """Main function for modular doubling test"""
    allocate(3, x)  # 3 qubits for mod 7 arithmetic

    # Initialize with test value: x=3
    x ^= 3

    # Perform modular doubling: x = (2*x) mod 7 = (2*3) mod 7 = 6
    modular_in_place_double(x, CURVE.p)

In [23]:
# Execute the quantum test
result = run_quantum_test(main, "Modular Double")

Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVbecumXyHyitgTSBIy1TqZw6c

🔧 Modular Double Circuit Metrics:
  Width (qubits): 4
Executing...

📊 Modular Double Test Results:
Quantum Result: [{'x': 6}: 2048]


### Modular Out-of-Place Multiplication

Computes $z = (x \times y) \bmod p$ where the result is stored in a separate register $z$.

In [24]:
# modular_out_of_place_multiply implementation and circuit synthesis
from classiq import (
    SIGNED,
    CInt,
    Output,
    QArray,
    QBit,
    QNum,
    X,
    allocate,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def modular_out_of_place_multiply(
    x: QArray[QBit], y: QArray[QBit], z: QArray[QBit], modulus: int
) -> None:
    """
    Reversible, out-of-place modular multiplication of two integers modulo a constant integer modulus.
    Given two n-bit integers x and y encoded in quantum bit arrays `x` and `y`, and a constant integer `modulus`,
    this function computes the product (x * y) mod modulus. The result is held in the third register `z`,
    which must be initialized to |0⟩.

    Algorithm (Double-and-Add for Multiplication):
    1. For each bit of x (from MSB to LSB):
       - If bit is 1: add y to accumulator z
       - Double z for next iteration (except last)
    2. Perform modular reduction at each step

    Args:
        x (QArray[QBit]): Quantum bit array encoding the first integer x.
        y (QArray[QBit]): Quantum bit array encoding the second integer y.
        z (QArray[QBit]): Quantum bit array for the result. Must be in state |0⟩ initially.
        modulus (int): Constant integer modulus.

    References:
        Based on Section 4.3.2 of Proos & Zalka: "Shor's discrete logarithm quantum algorithm
        for elliptic curves", https://arxiv.org/abs/quant-ph/0301141
    """
    n = x.size
    for idx in reversed(range(n)):
        # If x[idx] = 1, add y to z (controlled addition)
        control(x[idx], lambda: modular_in_place_add(y, z, modulus))
        # Double z for next iteration (except for the last iteration)
        if idx != 0:
            modular_in_place_double(z, modulus)

**Demonstrating modular out-of-place multiplication**: Multiply two quantum registers:
$$z \leftarrow (x \times y) \bmod 7 = (3 \times 5) \bmod 7 = 1$$

**Algorithm steps (Double-and-Add)**:
1. **Setup**: $x = 3 = 011_2$, $y = 5$, $z = 0$, modulus $= 7$
2. **Bit 2 (MSB)**: $x[2] = 0$ → skip addition, $z = 0$
   - Double: $z = 2 \times 0 = 0$
3. **Bit 1**: $x[1] = 1$ → add $y$: $z = (0 + 5) \bmod 7 = 5$
   - Double: $z = (2 \times 5) \bmod 7 = 3$
4. **Bit 0 (LSB)**: $x[0] = 1$ → add $y$: $z = (3 + 5) \bmod 7 = 1$
   - No doubling (last iteration)
5. **Result**: $z = 1$

In [25]:
@qfunc
def main(x: Output[QNum], y: Output[QNum], z: Output[QNum]) -> None:
    """Main function for modular multiplication test"""
    allocate(3, x)  # 3 qubits for mod 7 arithmetic
    allocate(3, y)  # 3 qubits for mod 7 arithmetic
    allocate(3, z)  # 3 qubits for result (initialized to 0)

    # Initialize with test values: x=3, y=5
    x ^= 3  # 011 in binary
    y ^= 5  # 101 in binary
    # z starts at 0 (000 in binary)

    # Perform modular multiplication: z = (x * y) mod 7 = (3 * 5) mod 7 = 1
    modular_out_of_place_multiply(x, y, z, CURVE.p)

In [26]:
# Execute the quantum test
result = run_quantum_test(main, "Modular Multiply")

Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVbh80yN9f7Qo3LjOTeytvMRi1

🔧 Modular Multiply Circuit Metrics:
  Width (qubits): 12
Executing...

📊 Modular Multiply Test Results:
Quantum Result: [{'x': 3, 'y': 5, 'z': 1}: 2048]


### Modular Square

Computes $z = x^2 \bmod p$ - optimized squaring operation.

In [27]:
# modular_square implementation and circuit synthesis
from classiq import (
    SIGNED,
    CInt,
    Output,
    QArray,
    QBit,
    QNum,
    X,
    allocate,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def modular_square(x: QArray[QBit], z: QArray[QBit], p: int) -> None:
    """
    Modular squaring of x modulo p, storing result in z.

    Implements x^2 mod p using controlled modular addition and doubling.
    The algorithm is based on the Q# ModularSquDblAddConstantModulus operation.

    Algorithm (Optimized Double-and-Add for Squaring):
    1. For each bit i of x (from MSB-1 down to 1):
       - If x[i] = 1: add x to accumulator z
       - Double z for next iteration
    2. For the LSB (bit 0):
       - If x[0] = 1: add x to accumulator z
       - No doubling after LSB
    3. All operations performed modulo p

    Args:
        x: Input quantum bit array to be squared
        z: Output quantum bit array to store x^2 mod p
        p: Classical modulus value

    Note: This is more efficient than general multiplication since we're
    multiplying x by itself, allowing for optimized bit processing.
    """
    n = x.size
    assert z.size == n, "Output register z must have the same number of qubits as x"

    anc = QBit("anc")
    allocate(1, anc)

    # Process bits from MSB-1 down to 1
    for i in range(n - 1, 0, -1):
        control(x[i], lambda: X(anc))
        control(anc, lambda: modular_in_place_add(x, z, p))
        control(x[i], lambda: X(anc))
        modular_in_place_double(z, p)

    # Process LSB (bit 0) - no doubling after this
    control(x[0], lambda: X(anc))
    control(anc, lambda: modular_in_place_add(x, z, p))
    control(x[0], lambda: X(anc))

    free(anc)

**Demonstrating modular squaring**: Square quantum register value:
$$z \leftarrow x^2 \bmod 7 = 3^2 \bmod 7 = 2$$

**Algorithm steps (Optimized Double-and-Add for Squaring)**:
1. **Setup**: $x = 3 = 011_2$, $z = 0$, modulus $= 7$
2. **Bit 1**: $x[1] = 1$ → add $x$: $z = (0 + 3) \bmod 7 = 3$
   - Double: $z = (2 \times 3) \bmod 7 = 6$
3. **Bit 0 (LSB)**: $x[0] = 1$ → add $x$: $z = (6 + 3) \bmod 7 = 2$
   - No doubling (final step)
4. **Result**: $z = 2$

In [28]:
@qfunc
def main(x: Output[QNum], z: Output[QNum]) -> None:
    """Main function for modular squaring test"""
    allocate(3, x)  # 3 qubits for mod 7 arithmetic
    allocate(3, z)  # 3 qubits for result (initialized to 0)

    # Initialize with test value: x=3
    x ^= 3  # 011 in binary
    # z starts at 0 (000 in binary)

    # Perform modular squaring: z = x^2 mod 7 = 3^2 mod 7 = 2
    modular_square(x, z, CURVE.p)

In [29]:
# Execute the quantum test
result = run_quantum_test(main, "Modular Square")

Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVbknuZPuXSLNS4OUb78DKMzuW

🔧 Modular Square Circuit Metrics:
  Width (qubits): 10
Executing...

📊 Modular Square Test Results:
Quantum Result: [{'x': 3, 'z': 2}: 2048]


### Modular Inverse Implementation

Computes $x^{-1} \bmod p$ such that $(x \times x^{-1}) \equiv 1 \pmod{p}$.

**Implementation Strategy**: For our educational demonstration on the small field $\mathbb{F}_7$, we use a simplified lookup table approach that allows us to:
- Execute the complete Shor's ECDLP algorithm on quantum simulators
- Understand the overall algorithm flow without getting bogged down in inverse computation details
- Focus on the elliptic curve arithmetic and period-finding aspects

**Scalable Implementation**: The mock implementation serves as a placeholder for the full quantum modular inverse algorithm. In practice, quantum elliptic curve cryptanalysis requires:
- **Kaliski's Algorithm**: A quantum-optimized modular inverse algorithm specifically designed for efficient quantum implementation

**Future Notebook**: A dedicated tutorial will cover the complete Kaliski algorithm implementation

For now, our lookup table approach enables us to demonstrate the complete Shor's algorithm pipeline and understand how modular inverse fits into the larger elliptic curve discrete logarithm solution.

In [30]:
# mock_kaliski_inverse_modulus_7 implementation and circuit synthesis
from classiq import (
    SIGNED,
    CInt,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    bind,
    control,
    create_model,
    free,
    inplace_add,
    qfunc,
    show,
    synthesize,
    within_apply,
)


@qfunc
def _set_value(reg: QNum, value: int) -> None:
    """Helper function to set a value in a quantum register."""
    reg ^= value


@qfunc
def mock_kaliski_inverse_modulus_7(x: QNum, result: QNum) -> None:
    """
    Mock implementation of Kaliski algorithm for modular inverse modulo 7.
    For input x in range 1-6, computes x^(-1) mod 7.
    The modular inverses modulo 7 are:
    1^(-1) = 1 mod 7
    2^(-1) = 4 mod 7
    3^(-1) = 5 mod 7
    4^(-1) = 2 mod 7
    5^(-1) = 3 mod 7
    6^(-1) = 6 mod 7

    Algorithm (Lookup Table using Controlled Operations):
    Uses a series of controlled operations to implement the inverse mapping.
    Each control checks for a specific input value and sets the corresponding inverse.

    Args:
        x (QNum): Input quantum number (should be in range 1-6)
        result (QNum): Output quantum number to store the inverse
    """
    # Use a series of controlled operations to set the correct inverse
    # For x = 1: result = 1
    control(x == 1, lambda: _set_value(result, 1))

    # For x = 2: result = 4
    control(x == 2, lambda: _set_value(result, 4))

    # For x = 3: result = 5
    control(x == 3, lambda: _set_value(result, 5))

    # For x = 4: result = 2
    control(x == 4, lambda: _set_value(result, 2))

    # For x = 5: result = 3
    control(x == 5, lambda: _set_value(result, 3))

    # For x = 6: result = 6
    control(x == 6, lambda: _set_value(result, 6))

**Demonstrating modular inverse lookup table**: Compute modular inverse using controlled operations:
$$\text{result} \leftarrow x^{-1} \bmod 7 = 3^{-1} \bmod 7 = 5$$

In [31]:
@qfunc
def main(x: Output[QNum], result: Output[QNum]) -> None:
    """Main function for modular inverse test"""
    allocate(3, x)  # 3 qubits for mod 7 arithmetic
    allocate(3, result)  # 3 qubits for result (inverse)
    # Initialize with test value: x=3
    x ^= 3
    # Perform modular inverse: result = x^(-1) mod 7 = 3^(-1) mod 7 = 5
    mock_kaliski_inverse_modulus_7(x, result)

In [32]:
# Execute the quantum test
result_counts = run_quantum_test(main, "Modular Inverse")

Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVbm0OBZ2qYRXApf5I6tA8Wsls

🔧 Modular Inverse Circuit Metrics:
  Width (qubits): 7
Executing...

📊 Modular Inverse Test Results:
Quantum Result: [{'x': 3, 'result': 5}: 2048]


## 6. Demonstrating Quantum Scalar Multiplication

Let's test `ell_mult_add` with a concrete example using our curve parameters and verify the results against classical computation.

**Test Setup**:
- **Initial point**: $P_0 = [4, 2]$ (our INITIAL_POINT)
- **Generator**: $G = [0, 5]$ (our GENERATOR_G)  
- **Scalar**: $k$ in quantum superposition over $\{0, 1, 2, ..., 7\}$
- **Operation**: Compute $P_0 + k \cdot G$ for all $k$ simultaneously

**Expected Results** (classical verification):
For each value of $k$, we should get $P_0 + k \cdot G$:
- $k=0$: $[4,2] + 0 \cdot [0,5] = [4,2]$
- $k=1$: $[4,2] + 1 \cdot [0,5] = [4,2] + [0,5] = ?$
- $k=2$: $[4,2] + 2 \cdot [0,5] = [4,2] + [6,2] = ?$
- And so on...

The quantum circuit will compute all these results in parallel through superposition!

**Classical Verification Setup**: We'll install and use the `tinyec` library to verify our quantum results against classical elliptic curve computations. This lightweight library provides the classical elliptic curve arithmetic needed to confirm our quantum algorithm produces correct results.

In [33]:
! pip install tinyec


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [34]:
# Demonstrate ell_mult_add with verification
from tinyec.ec import Curve, Point, SubGroup


# Classical verification using tinyec library
def verify_ell_mult_add_results(quantum_results, initial_ecp, G, curve_params):
    """
    Verify quantum results against classical elliptic curve computation using tinyec.
    """
    print("=== Classical Verification using TinyEC ===")

    # Setup tinyec curve
    field = SubGroup(p=curve_params.p, g=(G[0], G[1]), n=10, h=None)
    tinyec_curve = Curve(a=curve_params.a, b=curve_params.b, field=field, name="custom")

    # Convert points to tinyec format
    initial_ecp_tinyec = Point(tinyec_curve, initial_ecp[0], initial_ecp[1])
    G_tinyec = Point(tinyec_curve, G[0], G[1])

    print(f"Initial point: ({initial_ecp_tinyec.x}, {initial_ecp_tinyec.y})")
    print(f"Generator G: ({G_tinyec.x}, {G_tinyec.y})")
    print(
        f"Curve: y² = x³ + {curve_params.a}x + {curve_params.b} (mod {curve_params.p})"
    )
    print()

    # Verify each k value
    matches = 0
    total = 0

    for result in quantum_results:
        k_val = result.state.get("k", 0)  # Extract k value from quantum result
        quantum_x = result.state.get("x", 0)  # Extract resulting x-coordinate
        quantum_y = result.state.get("y", 0)  # Extract resulting y-coordinate

        # Classical computation: P_0 + k*G
        try:
            k_G = k_val * G_tinyec  # Compute k*G classically
            classical_result = initial_ecp_tinyec + k_G  # Add to initial point

            print(f"k={k_val}:")
            print(f"  Quantum result:  ({quantum_x}, {quantum_y})")
            print(f"  Classical result: ({classical_result.x}, {classical_result.y})")

            if quantum_x == classical_result.x and quantum_y == classical_result.y:
                print(f"  ✅ MATCH")
                matches += 1
            else:
                print(f"  ❌ MISMATCH")

            total += 1
            print()

        except Exception as e:
            print(f"k={k_val}: Error in classical computation: {e}")

    print(
        f"Verification Summary: {matches}/{total} results match classical computation"
    )
    return matches == total


@qfunc
def main(x: Output[QNum], y: Output[QNum], k: Output[QNum]) -> None:
    """Main function for ell_mult_add test"""
    allocate(3, x)  # EC point x-coordinate
    allocate(3, y)  # EC point y-coordinate
    allocate(3, k)  # Scalar in superposition (0 to 7)

    # Initialize point to INITIAL_POINT [4, 2]
    x ^= INITIAL_POINT[0]  # x = 4
    y ^= INITIAL_POINT[1]  # y = 2

    # Create superposition over all possible k values (0 to 7)
    hadamard_transform(k)

    # Perform quantum scalar multiplication: (x,y) = INITIAL_POINT + k*GENERATOR_G
    ell_mult_add(x, y, k, GENERATOR_G, CURVE.p, CURVE.a, CURVE.b)


# Execute the quantum test
print("🚀 Executing quantum scalar multiplication test...")
result_counts = run_quantum_test(main, "Elliptic Curve Scalar Multiplication")

print(f"\n📊 Quantum Execution Results:")
print(f"Total measurement outcomes: {len(result_counts)}")
print(f"Each outcome represents: INITIAL_POINT + k*GENERATOR_G for different k values")

# Verify against classical computation
print(f"\n🔍 Verifying against classical elliptic curve arithmetic...")
all_correct = verify_ell_mult_add_results(
    result_counts, INITIAL_POINT, GENERATOR_G, CURVE
)

if all_correct:
    print("🎉 All quantum results verified against classical computation!")
    print("✅ Quantum scalar multiplication working correctly")
else:
    print("⚠️  Some results don't match - check implementation")

🚀 Executing quantum scalar multiplication test...
Synthesizing...
Quantum program link: https://platform.classiq.io/circuit/2zVdlA9aH1nI2jLqISYiptf5Pqg

🔧 Elliptic Curve Scalar Multiplication Circuit Metrics:
  Width (qubits): 22
Executing...

📊 Elliptic Curve Scalar Multiplication Test Results:
Quantum Result: [{'x': 3, 'y': 2, 'k': 7}: 276, {'x': 4, 'y': 2, 'k': 0}: 269, {'x': 3, 'y': 2, 'k': 2}: 264, {'x': 3, 'y': 5, 'k': 4}: 261, {'x': 5, 'y': 0, 'k': 3}: 253, {'x': 4, 'y': 5, 'k': 1}: 248, {'x': 4, 'y': 2, 'k': 5}: 247, {'x': 4, 'y': 5, 'k': 6}: 230]

📊 Quantum Execution Results:
Total measurement outcomes: 8
Each outcome represents: INITIAL_POINT + k*GENERATOR_G for different k values

🔍 Verifying against classical elliptic curve arithmetic...
=== Classical Verification using TinyEC ===
Initial point: (4, 2)
Generator G: (0, 5)
Curve: y² = x³ + 5x + 4 (mod 7)

k=7:
  Quantum result:  (3, 2)
  Classical result: (3, 2)
  ✅ MATCH

k=0:
  Quantum result:  (4, 2)
  Classical result: (

## 7. Execute the Full Algorithm

Now we'll execute the complete Shor's ECDLP algorithm following the `run_shor_quantum` logic. The execution pipeline consists of several stages:

### Execution Pipeline

1. **Model Creation**: Convert our quantum functions into a Classiq quantum model
2. **Synthesis**: Optimize and compile the model into executable quantum program
3. **Quantum Execution**: Run the quantum program on a quantum simulator 
4. **Result Collection**: Gather measurement outcomes from all quantum registers

### Circuit Optimization

We use several optimization strategies:
- **Width Optimization**: Minimize the number of qubits required
- **Depth Optimization**: Reduce circuit depth for better fidelity
- **Gate Count Minimization**: Reduce total number of quantum operations



In [35]:
# Complete execution pipeline following run_shor_quantum logic
from collections import defaultdict

from classiq import write_qmod


@qfunc
def main(
    ecp_x: Output[QNum], ecp_y: Output[QNum], x1: Output[QNum], x2: Output[QNum]
) -> None:
    main_shor_ecdlp(ecp_x, ecp_y, x1, x2)


def create_shor_quantum_program():
    """
    Create and synthesize the Shor's ECDLP quantum program.
    Returns the quantum program for reuse in analysis.
    """
    print("🔧 Creating model for Shor's ECDLP algorithm...")

    constraints = Constraints(optimization_parameter="width")
    preferences = Preferences(timeout_seconds=3600, optimization_level=1)
    qmod = create_model(main, constraints=constraints, preferences=preferences)
    write_qmod(qmod, "elliptic_curve_discrete_log", symbolic_only=False)

    print("🔧 Synthesizing quantum program...")
    qprog = synthesize(qmod)

    # Display circuit metrics
    num_qubits = qprog.data.width
    if hasattr(qprog.data, "circuit_depth"):
        print(f"📏 Circuit depth: {qprog.data.circuit_depth}")
    elif hasattr(qprog, "transpiled_circuit") and hasattr(
        qprog.transpiled_circuit, "depth"
    ):
        print(f"📏 Circuit depth: {qprog.transpiled_circuit.depth}")
    else:
        print("⚠️  Depth attribute not found")

    print(f"📏 Number of qubits: {num_qubits}")
    show(qprog)

    return qprog


def execute_and_analyze_results(qprog):
    """
    Execute the quantum circuit and analyze the measurement results.
    """
    print("🚀 Executing Shor's ECDLP with QFT...")

    # Execute the quantum circuit
    result = execute(qprog).result()
    print("✅ Execution complete.")

    # Parse quantum results
    quantum_results_raw = result[0].value.parsed_counts
    raw_data = str(quantum_results_raw)

    # Extract measurement data
    pattern = r"(\{[^\}]+\}): (\d+)"
    matches = re.findall(pattern, raw_data)

    quantum_results = []
    for dict_str, count in matches:
        d = ast.literal_eval(dict_str)
        d["count"] = int(count)
        quantum_results.append(d)

    # Analyze (x1, x2) pairs from quantum results (post-QFT)
    distinct_pairs = set()
    pair_counts = defaultdict(int)
    total_counts = 0

    for res in quantum_results:
        x1 = res["x1"]
        x2 = res["x2"]
        count = res["count"]
        pair = (x1, x2)
        distinct_pairs.add(pair)
        pair_counts[pair] += count
        total_counts += count

    # Sort pairs by probability (descending)
    sorted_pairs = sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)

    # Display results
    print(f"\n📊 Quantum Execution Results:")
    print(f"Total distinct (x1, x2) pairs: {len(distinct_pairs)}")
    print(f"Total measurements: {total_counts}")

    print("\nTop 10 distinct pairs sorted by probability (descending):")
    for pair, count in sorted_pairs[:10]:
        probability = count / total_counts
        print(f"  {pair}: count={count}, probability={probability:.4f}")

    return sorted_pairs, total_counts, quantum_results_raw


def test_shor_ecdlp():
    """
    Complete pipeline: create quantum program, execute, and analyze results.
    Returns quantum program object for further analysis.
    """
    # Create the quantum program
    qprog = create_shor_quantum_program()

    # Execute and analyze
    sorted_pairs, total_counts, raw_results = execute_and_analyze_results(qprog)

    return sorted_pairs, total_counts, qprog, raw_results


# Execute the complete algorithm
sorted_pairs, total_counts, qprog, raw_results = test_shor_ecdlp()

🔧 Creating model for Shor's ECDLP algorithm...
🔧 Synthesizing quantum program...
📏 Circuit depth: 101179
📏 Number of qubits: 25
Quantum program link: https://platform.classiq.io/circuit/2zVhaowDOdtsr18K8YzQkMeEU0m
🚀 Executing Shor's ECDLP with QFT...
✅ Execution complete.

📊 Quantum Execution Results:
Total distinct (x1, x2) pairs: 62
Total measurements: 2048

Top 10 distinct pairs sorted by probability (descending):
  (0, 0): count=398, probability=0.1943
  (5, 5): count=334, probability=0.1631
  (3, 3): count=325, probability=0.1587
  (6, 6): count=132, probability=0.0645
  (2, 2): count=131, probability=0.0640
  (6, 7): count=65, probability=0.0317
  (2, 1): count=61, probability=0.0298
  (7, 6): count=54, probability=0.0264
  (1, 2): count=52, probability=0.0254
  (1, 1): count=32, probability=0.0156


### Circuit Analysis

We can analyze the transpiled circuit characteristics from the quantum program object:

In [36]:
qprog.transpiled_circuit.count_ops

{'u': 94307, 'cx': 89784}

## 7. Post-Processing Results

After quantum execution, we need to extract the discrete logarithm from the measurement results. 


### Extract Discrete Logarithm
From the valid solutions, we solve for the discrete logarithm using:
$$l = y_2 \cdot y_1^{-1} \bmod r$$
where $r$ is the order of the generator point $G = [0, 5]$, which is $r = 5$ in our case.

**Note**: The order $r$ is the smallest positive integer such that $r \cdot G = \mathcal{O}$ (point at infinity). 



In [37]:
# Manual Discrete Logarithm Calculation for Top Valid Pairs
from tinyec.ec import Curve, Point, SubGroup

print("🔍 MANUAL DISCRETE LOGARITHM CALCULATION")
print("=" * 50)

# Generator order and modular inverse table for F_5
generator_order = 5
print(f"Generator order r = {generator_order}")
print(f"Modular inverses mod 5: 1⁻¹=1, 2⁻¹=3, 3⁻¹=2, 4⁻¹=4")
print()

# Calculate l for pair (3, 3)
print("📊 Pair (y₁=3, y₂=3) - probability=0.1543")
y1, y2 = 3, 3
y1_inverse = 2  # 3^(-1) = 2 mod 5
l_1 = (-y2 * y1_inverse) % generator_order  # Note the negative sign
print(
    f"  l = -y₂ × y₁⁻¹ = -{y2} × {y1_inverse} = {-y2 * y1_inverse} ≡ {l_1} (mod {generator_order})"
)
print(f"  Result: l = {l_1}")
print()

# Calculate l for pair (2, 2)
print("📊 Pair (y₁=2, y₂=2) - probability=0.0625")
y1, y2 = 2, 2
y1_inverse = 3  # 2^(-1) = 3 mod 5
l_2 = (-y2 * y1_inverse) % generator_order  # Note the negative sign
print(
    f"  l = -y₂ × y₁⁻¹ = -{y2} × {y1_inverse} = {-y2 * y1_inverse} ≡ {l_2} (mod {generator_order})"
)
print(f"  Result: l = {l_2}")
print()

print("🎯 DISCRETE LOGARITHM CANDIDATES:")
print(f"  • l = {l_1} (from pair (3,3))")
print(f"  • l = {l_2} (from pair (2,2))")
print()

🔍 MANUAL DISCRETE LOGARITHM CALCULATION
Generator order r = 5
Modular inverses mod 5: 1⁻¹=1, 2⁻¹=3, 3⁻¹=2, 4⁻¹=4

📊 Pair (y₁=3, y₂=3) - probability=0.1543
  l = -y₂ × y₁⁻¹ = -3 × 2 = -6 ≡ 4 (mod 5)
  Result: l = 4

📊 Pair (y₁=2, y₂=2) - probability=0.0625
  l = -y₂ × y₁⁻¹ = -2 × 3 = -6 ≡ 4 (mod 5)
  Result: l = 4

🎯 DISCRETE LOGARITHM CANDIDATES:
  • l = 4 (from pair (3,3))
  • l = 4 (from pair (2,2))



### Classical Verification

Finally, we verify each candidate solution by checking:
$$ l \cdot G \stackrel{?}{=} P_{target}$$

using classical elliptic curve arithmetic.

In [38]:
# Classical verification using tinyec
print("🧪 CLASSICAL VERIFICATION using TinyEC:")
print(f"Testing: l × {GENERATOR_G} = {TARGET_POINT}")
print()

# Setup tinyec curve
field = SubGroup(p=CURVE.p, g=(GENERATOR_G[0], GENERATOR_G[1]), n=10, h=None)
tinyec_curve = Curve(a=CURVE.a, b=CURVE.b, field=field, name="custom")

# Convert points to tinyec format
generator_tinyec = Point(tinyec_curve, GENERATOR_G[0], GENERATOR_G[1])
target_ecp_tinyec = Point(tinyec_curve, TARGET_POINT[0], TARGET_POINT[1])

print(f"TinyEC curve setup:")
print(f"  Generator G: ({generator_tinyec.x}, {generator_tinyec.y})")
print(f"  Target point: ({target_ecp_tinyec.x}, {target_ecp_tinyec.y})")
print()

for l in [l_1, l_2]:
    print(f"Testing l = {l}:")

    # Compute l * G using tinyec
    l_times_G = l * generator_tinyec
    print(f"  {l} × G = ({l_times_G.x}, {l_times_G.y})")
    print(f"  Target: ({target_ecp_tinyec.x}, {target_ecp_tinyec.y})")

    if l_times_G.x == target_ecp_tinyec.x and l_times_G.y == target_ecp_tinyec.y:
        print(f"  🎉 CORRECT! l = {l} is the discrete logarithm")
        print(f"  ✅ Verification: {l} × {GENERATOR_G} = {TARGET_POINT}")
    else:
        print(
            f"  ❌ Incorrect: got ({l_times_G.x}, {l_times_G.y}) instead of ({target_ecp_tinyec.x}, {target_ecp_tinyec.y})"
        )
    print()

🧪 CLASSICAL VERIFICATION using TinyEC:
Testing: l × [0, 5] = [0, 2]

TinyEC curve setup:
  Generator G: (0, 5)
  Target point: (0, 2)

Testing l = 4:
  4 × G = (0, 2)
  Target: (0, 2)
  🎉 CORRECT! l = 4 is the discrete logarithm
  ✅ Verification: 4 × [0, 5] = [0, 2]

Testing l = 4:
  4 × G = (0, 2)
  Target: (0, 2)
  🎉 CORRECT! l = 4 is the discrete logarithm
  ✅ Verification: 4 × [0, 5] = [0, 2]



### Algorithm Complexity Summary

| Method | Time Complexity | Space (Qubits) | Example (256-bit) |
|--------|-----------------|----------------|-------------------|
| Classical (Pollard's rho) | $O(\sqrt{n})$ | $O(\log n)$ | $\approx 2^{128}$ ops |
| Quantum (Shor) | $O((\log n)^3)$ | $O(\log n)$ | $\approx 2^{24}$ ops |

This exponential quantum speedup is what makes Shor's algorithm a potential threat to elliptic curve cryptography used in modern security systems.

## References

<a id="ref1"></a>
[1] Martin Roetteler, Michael Naehrig, Krysta M. Svore, and Kristin Lauter. "Quantum resource estimates for computing elliptic curve discrete logarithms." *arXiv preprint arXiv:1706.06752* (2017). [https://arxiv.org/pdf/1706.06752](https://arxiv.org/pdf/1706.06752)

[2] Diego Polimeni and Raphael Seidel. "End-to-end compilable implementation of quantum elliptic curve logarithm in Qrisp." *arXiv preprint arXiv:2501.10228v1* (2025). [https://arxiv.org/abs/2501.10228](https://arxiv.org/abs/2501.10228)

[3] Peter W. Shor. "Polynomial-time algorithms for prime factorization and discrete logarithms on a quantum computer." *SIAM Journal on Computing*, 26(5):1484-1509 (1997).